In [1]:
def prefilter_items(data):
    # Уберем самые популярные товары (их и так купят)
    popularity = data_train.groupby('item_id')['user_id'].nunique().reset_index() / data_train['user_id'].nunique()
    popularity.rename(columns={'user_id': 'share_unique_users'}, inplace=True)
    
    top_popular = popularity[popularity['share_unique_users'] > 0.5].item_id.tolist()
    data = data[~data['item_id'].isin(top_popular)]
    
    # Уберем самые НЕ популярные товары (их и так НЕ купят)
    top_notpopular = popularity[popularity['share_unique_users'] < 0.01].item_id.tolist()
    data = data[~data['item_id'].isin(top_notpopular)]
    
    # Уберем товары, которые не продавались за последние 12 месяцев
    old_items = data.groupby(by=['item_id'])['item_id', 'week_no'].max()
    old_items = list(old_items[old_items['week_no'] <= (data['week_no'].max() - 52)].index)
    data = data[~data['item_id'].isin(old_items)]
    # Уберем не интересные для рекоммендаций категории (department)
    # к примеру уберем товары купленные менее 500 раз
    top_departments = data['department'].value_counts()[lambda x: x > 500].index
    data = data[data['department'].isin(top_departments)]
    
    # Уберем слишком дешевые товары (на них не заработаем). 1 покупка из рассылок стоит 60 руб. допустим примерно 10 центов
    data['price'] = data['sales_value'] / (np.maximum(data['quantity'], 1))
    data = data[data['price'] > 0.1]
    # Уберем слишком дорогие товары
    data = data[data['price'] < data['price'].quantile(0.99995)]
    # ...
    
def postfilter_items(user_id, recommednations):
    pass

1) Попытаться ответить на вопросы/выдвинуть гипотезы


### А точно нужно сортировать по вероятности?

Мы не считаем вероятности. Мы можем поделить товары на категории и предложить популярное из той категории, которой пользователь интересуется. Либо проаналиpировать те товары, которые покупатель преобретал, и предложить то, что преобретают другие покупатели с такими же товарами в корзине.

### Какую метрику использовать?

Для оценки качества следует пользоваться метрикой precision.

### Сколько раз в неделю отправляем рассылку?

Тут может быть несколько вариантов.
Когда пользователь только зарегистировался, то практически сразу ему стоит, что нибудь предложить. А дальше рассылка может идти по мере потери интереса у пользователя, например он стал реже посещать или стал реже покупать товары, то стоит сделать ему рассылку. Или это должна быть некоторая триггерная схема, какой то интересные товар стали чаще покупать, если пользователь его не преобретал, то стоит ему предложить этот товар.

### В какое время отправляем рассылку?

С учетом часовых поясов, в не рабочее время для пользователя.

### Будем отправлять одному юзеру много раз наши рекоммендации. Как добиться того, чтобы они хоть немного отличались?

Делать проверку этих рекомендаций с тем что было уже отправлено, фиксировать когда была сделана предыдущая отправка пользователю.

### Нужно ли, чтобы в одной рассылке были разные товары? Как определить, что товары разные? Как добиться того, чтобы они были разными?

Нужно. Следует изначально разбить товары на категории, предложить сначала то, что попало в топ, далее посмотреть, что еще покупали клиенты, которые интересовались данным товаром из уже предложенного.